# Day 18

https://adventofcode.com/2022/day/18

Part 1 i figured out myself and it was quite easy. For Part 2 I got some inspiration from Hyper-Neutrino (maybe I could have figured it out myself, in the end it was not so hard):

https://github.com/hyper-neutrino/advent-of-code/blob/main/2022/day18p2.py

## Part 1

In [ ]:
test_data = """\
Blueprint 1:
  Each ore robot costs 4 ore.
  Each clay robot costs 2 ore.
  Each obsidian robot costs 3 ore and 14 clay.
  Each geode robot costs 2 ore and 7 obsidian.

Blueprint 2:
  Each ore robot costs 2 ore.
  Each clay robot costs 3 ore.
  Each obsidian robot costs 3 ore and 8 clay.
  Each geode robot costs 3 ore and 12 obsidian."""

# format into actual input data format
blocks = test_data.split("\n\n")
test_data = ""
for block in blocks:
    lines = block.split()
    test_data += " ".join([line.strip() for line in lines])
    test_data += "\n"
test_data = test_data.rstrip()
#print(test_data)

with open("day19.txt") as f:
    inp_data = f.read()

In [ ]:
from collections import deque
import time

print("\nPart 1")

def solution1(data):

    sol1 = 0

    _t = time.time()
    for i, bp in enumerate(data.split("\n")):
        __t = time.time()
        count = 0

        infos = bp.split(": ")[1].split(". ")
        cost_ore_robot = int(infos[0].split()[-2])
        cost_clay_robot = int(infos[1].split()[-2])    
        cost_obsidian_robot = [int(x) for x in [infos[2].split()[i] for i in [-5, -2]]]
        cost_geode_robot = [int(x) for x in [infos[3].split()[i] for i in [-5, -2]]]

        ore_needed = cost_geode_robot[0] + cost_obsidian_robot[0]
        clay_needed = cost_obsidian_robot[1]

        maxval = 0
        remtime = 24

        Q = deque([(
            0, 0, 0, 0, 
            1, 0, 0, 0, 
            remtime
            )])
        seen = set()

        print(f"Blueprint {i+1}")
        dfs = False
        while Q:
            # pop() for DFS and popleft for BFS

            if dfs:
                key = Q.pop()
            else:            
                key = Q.popleft()
                if key in seen:
                    continue

                seen.add(key)

            (ore, clay, obsidian, geode, r_ore, r_clay, r_obsidian, r_geode, remtime) = key
            count += 1

            if count % 1000000 == 0 and not dfs:
                print(count, (ore, clay, obsidian, geode), (r_ore, r_clay, r_obsidian, r_geode), remtime)

            if remtime == 0:
                maxval = max(maxval, geode)     
                continue
            remtime -= 1

            if remtime <= 2 and not dfs:
                print("switch to dfs")
                dfs = True


            if ore >= cost_geode_robot[0] and obsidian >= cost_geode_robot[1]:
                # always produce geode robot     
                Q.append(
                    (ore + r_ore - cost_geode_robot[0], clay + r_clay, obsidian + r_obsidian - cost_geode_robot[1], geode + r_geode, 
                     r_ore, r_clay, r_obsidian, r_geode + 1, 
                     remtime)
                ) 
                continue

            # branch: build no robot
            #if not build_sth:
            Q.append(
                    (ore + r_ore, clay + r_clay, obsidian + r_obsidian, geode + r_geode, 
                    r_ore, r_clay, r_obsidian, r_geode, 
                    remtime)
                )    


            # branch: build ore robot
            if r_ore < ore_needed:
                if ore >= cost_ore_robot:
                    Q.append(
                        (ore + r_ore - cost_ore_robot, clay + r_clay, obsidian + r_obsidian, geode + r_geode, 
                         r_ore + 1, r_clay, r_obsidian, r_geode, 
                         remtime)
                    )

            # branch: build clay
            if r_clay < clay_needed:
                if ore >= cost_clay_robot:
                    Q.append(
                        (ore + r_ore - cost_clay_robot, clay + r_clay, obsidian + r_obsidian, geode + r_geode, 
                         r_ore, r_clay + 1, r_obsidian, r_geode, 
                         remtime)
                    )

            # branch: build obsidian robot
            if r_obsidian < cost_geode_robot[1]:
                if ore >= cost_obsidian_robot[0] and clay >= cost_obsidian_robot[1]:
                    Q.append(
                        (ore + r_ore - cost_obsidian_robot[0], clay + r_clay - cost_obsidian_robot[1], obsidian + r_obsidian, geode + r_geode, 
                         r_ore, r_clay, r_obsidian + 1, r_geode, 
                         remtime)
                    ) 
                    build_sth = True

        print(f"processed {count:.3e} queue items in {time.time() - __t:.2f} s")
        print(f"maximum value: {maxval}")
        sol1 += maxval * (i + 1)
    
    print(f"total elapsed time: {time.time() - _t:.2f} s")
    print("Solution:", sol1)
    return sol1

print("test data")
# takes 23s with pypy
sol1 = solution1(test_data)
assert sol1 == 33
print("test passed")

In [ ]:
print("\ninput data")
# takes 73s with pypy
sol1 = solution1(inp_data)
assert sol1 == 1487
print("test passed")

## Part 2

In [ ]:
print("\nPart 2")

def solution2(data):

    sol2 = 1

    _t = time.time()
    for i, bp in enumerate(data.split("\n")[:3]):
        __t = time.time()
        count = 0

        infos = bp.split(": ")[1].split(". ")
        cost_ore_robot = int(infos[0].split()[-2])
        cost_clay_robot = int(infos[1].split()[-2])    
        cost_obsidian_robot = [int(x) for x in [infos[2].split()[i] for i in [-5, -2]]]
        cost_geode_robot = [int(x) for x in [infos[3].split()[i] for i in [-5, -2]]]

        ore_needed = max([cost_geode_robot[0], cost_obsidian_robot[0], cost_ore_robot, cost_clay_robot])
        clay_needed = cost_obsidian_robot[1]

        maxval = 0
        remtime = 32

        Q = deque([(
            0, 0, 0, 0, 
            1, 0, 0, 0, 
            remtime
            )])
        seen = set()

        print(f"Blueprint {i+1}")
        dfs = False
        while Q:


            # pop() for DFS and popleft for BFS

            if dfs:
                key = Q.pop()
            else:            
                key = Q.popleft()
                
                if key in seen:
                    continue
                seen.add(key)

            (ore, clay, obsidian, geode, r_ore, r_clay, r_obsidian, r_geode, remtime) = key
            count += 1

            if count % (1000000 * 100**int(dfs)) == 0:
                if not dfs and len(Q) > 1.5e7:
                    print("switch to dfs")
                    dfs = True
                    seen = set()
                print(f"{time.time() - __t:.2f}, {len(Q):.3e}, {count:.3e}", 
                      (ore, clay, obsidian, geode), 
                      (r_ore, r_clay, r_obsidian, r_geode), remtime)

            if remtime == 0:
                maxval = max(maxval, geode)     
                continue
            remtime -= 1



            # branch: build no robot
            #if not build_sth:


            if ore >= cost_geode_robot[0] and obsidian >= cost_geode_robot[1]:
                # always produce geode robot     
                Q.append(
                    (ore + r_ore - cost_geode_robot[0], clay + r_clay, obsidian + r_obsidian - cost_geode_robot[1], geode + r_geode, 
                     r_ore, r_clay, r_obsidian, r_geode + 1, 
                     remtime)
                ) 
                continue

            Q.append(
                    (ore + r_ore, clay + r_clay, obsidian + r_obsidian, geode + r_geode, 
                    r_ore, r_clay, r_obsidian, r_geode, 
                    remtime)
                )    


            # branch: build ore robot
            if r_ore < ore_needed:
                if ore >= cost_ore_robot:
                    Q.append(
                        (ore + r_ore - cost_ore_robot, clay + r_clay, obsidian + r_obsidian, geode + r_geode, 
                         r_ore + 1, r_clay, r_obsidian, r_geode, 
                         remtime)
                    )

            # branch: build clay
            if r_clay < clay_needed:
                if ore >= cost_clay_robot:
                    Q.append(
                        (ore + r_ore - cost_clay_robot, clay + r_clay, obsidian + r_obsidian, geode + r_geode, 
                         r_ore, r_clay + 1, r_obsidian, r_geode, 
                         remtime)
                    )

            # branch: build obsidian robot
            if r_obsidian < cost_geode_robot[1]:
                if ore >= cost_obsidian_robot[0] and clay >= cost_obsidian_robot[1]:
                    Q.append(
                        (ore + r_ore - cost_obsidian_robot[0], clay + r_clay - cost_obsidian_robot[1], obsidian + r_obsidian, geode + r_geode, 
                         r_ore, r_clay, r_obsidian + 1, r_geode, 
                         remtime)
                    ) 
                    build_sth = True

        print(f"processed {count:.3e} queue items in {time.time() - __t:.2f} s")
        print(f"maximum value: {maxval}")
        sol2 *= maxval
        
    print(f"total elapsed time: {time.time() - _t:.2f} s")
    print("Solution:", sol2)
    return sol2

print("test data")
# takes 293s with pypy
sol2 = solution2(test_data)
assert sol2 == 56 * 62
print("test passed")

In [ ]:
print("\ninput data")
# takes 402s with pypy
sol2 = solution2(inp_data)
assert sol2 == 13440
print("test passed")